# **차량 공유업체의 차량 파손 여부 분류하기**

## 0.미션

* 1) 미션1 : Data Preprocessing
    - **과제 수행 목표**
        - 본인의 구글 드라이브에 모델링 수행을 위해 적절한 폴더 및 파일로 **일관성 있게 정리**해야 합니다.
        - 제공된 데이터 : Car_Images.zip
            * Car_Images : 차량의 정상/파손 이미지 무작위 수집

* 2) 미션2 : CNN 모델링
    - **과제 수행 목표**
        - Tensorflow Keras를 이용하여 모델을 3개 이상 생성하세요.
            - 모델 구조와 파라미터는 자유롭게 구성하세요.
            - 단, 세부 목차에서 명시한 부분은 지켜주세요.

* 3) 미션3 : Data Argumentation & Transfer Learning
    - **과제 수행 목표**
        - 성능 개선을 위해 다음의 두가지를 시도하세요.
            * Data Augmentation을 적용하세요.(Image Generator)
            * Transfer Learning(VGG16)


## 1.환경설정 

### (1) 데이터셋 폴더 생성
- **세부요구사항**
    - C드라이브에 Datasets라는 폴더를 만드세요.
        - 구글드라이브를 사용하는경우 드라이브 첫 화면에 Datasets 라는 폴더를 만드세요. ('/content/drive/MyDrive/Datasets/')
    - 해당 폴더 안에 Car_Images.zip 파일을 넣으세요.

* 구글 Colab을 이용하는 경우

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### (2) 데이터셋 불러오기 
- **세부요구사항**
    - Car_Images.zip 파일을 C:/Datasets/ 경로에 압축 해제합니다.
    - zipfile 모듈을 이용하거나 다른 방식을 사용해도 됩니다.
        - 참고 자료 : [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 폴더구조(로컬)
        * C:/Datasets/ : 압축파일
        * C:/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 폴더구조(구글드라이브브)
        * /content/drive/MyDrive/Datasets/ : 압축파일
        * /content/drive/MyDrive/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 압축을 해제하면 다음과 같은 두 하위 폴더가 생성됩니다.
        * normal, abnormal : 각 폴더에는 이미지들이 있습니다.
        * 이후 단계에서 해당 경로로 부터 validation, test 셋을 추출하게 됩니다.
        

In [ ]:
import zipfile

In [ ]:
# 압축파일 경로
# 구글 드라이브인 경우 경로에 맞게 지정하세요.
dataset_path  = '/content/drive/MyDrive/Datasets/'
# dataset_path = 'C:/Datasets/'

file_path = dataset_path + 'Car_Images.zip'

In [ ]:
# 압축 해제
# data = zipfile.ZipFile(file_path)
# data.extractall( '/content/drive/MyDrive/Datasets/')

!unzip /content/drive/MyDrive/Datasets/Car_Images.zip -d /content/drive/MyDrive/Datasets/Car_Images_train/

### (3) 이미지 저장을 위한 폴더 생성
- **세부요구사항**
    - train, validation, test 을 위해 각각 하위 폴더 normal과 abnormal를 준비합니다.
        - train
            * 정상 이미지 저장소 : C:/Datasets/Car_Images_train/normal/ 
                * 구글드라이브 :   /content/drive/MyDrive/Datasets/Car_Images_train/normal/
            * 파손 이미지 저장소 : C:/Datasets/Car_Images_train/abnormal/
                * 구글드라이브 : /content/drive/MyDrive/Datasets/Car_Images_train/abnormal/
        - val, test 역시 동일한 구조로 생성합니다.
    - 직접 탐색기에서 폴더를 생성할 수도 있고, os 모듈을 이용하여 코드로 작성할 수도 있습니다.
        - 참고 자료 : [os document](https://docs.python.org/3/library/os.html)

In [ ]:

# 각각 경로 지정
import os 


# train 폴더는 압축을 해제하면서 이미 생성 되어 있습니다.

# test 폴더 만들기 os.mkdir()
os.mkdir('/content/drive/MyDrive/Datasets/Car_Images_test')
os.mkdir('/content/drive/MyDrive/Datasets/Car_Images_test/normal')
os.mkdir('/content/drive/MyDrive/Datasets/Car_Images_test/abnormal')

# validation 폴더 만들기
os.mkdir('/content/drive/MyDrive/Datasets/Car_Images_validation')
os.mkdir('/content/drive/MyDrive/Datasets/Car_Images_validation/normal')
os.mkdir('/content/drive/MyDrive/Datasets/Car_Images_validation/abnormal')

## 2.데이터 전처리

### (1) 데이터 분할 : Training set | Validation set | Test set 생성
- **세부요구사항**
    - Training set, Validation set, Test set을 만듭니다.
        * size
            * test : 전체에서 20%를 추출합니다.
            * validation : test를 떼어낸 나머지에서 다시 20%를 추출합니다.
        * 데이터는 랜덤하게 추출해야 합니다.
            - random, shutil 모듈을 이용하여 랜덤하게 추출할 수 있습니다.
                - [random document](https://docs.python.org/3/library/random.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
            * 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

#### 1) test, validation 크기를 지정

In [ ]:
import random, shutil

In [ ]:
tr_n_path = '/content/drive/MyDrive/Datasets/Car_Images_train/normal/'
tr_ab_path = '/content/drive/MyDrive/Datasets/Car_Images_train/abnormal/'

In [ ]:
# 전체 이미지 갯수를 확인합니다.
len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path))

(302, 303)

In [ ]:
# test 사이즈 : 전체 이미지의 20%
te_data_num = [round(len(os.listdir(tr_n_path))*0.2), round(len(os.listdir(tr_ab_path))*0.2)]
print(te_data_num)

# validation 사이즈 : test를 제외한 나머지 중에서 20%
val_data_num = [ round((len(os.listdir(tr_n_path))-te_data_num[0])*0.2) , round((len(os.listdir(tr_n_path))-te_data_num[1])*0.2) ]
print(val_data_num)

# train 사이즈
train_data_num = [len(os.listdir(tr_n_path)) - te_data_num[0] - val_data_num[0],
                  len(os.listdir(tr_ab_path))- te_data_num[1] - val_data_num[1]]

[60, 61]
[48, 48]


#### 2) test 셋 추출

In [ ]:
random.seed(2023)
# tr_n_path = '/content/drive/MyDrive/Datasets/Car_Images_train/normal/'
# tr_ab_path = '/content/drive/MyDrive/Datasets/Car_Images_train/abnormal/'

# Set the directory containing the files

# Get a list of all the files in the directory
n_files = os.listdir(tr_n_path)
ab_files = os.listdir(tr_ab_path)
# Choose a random file from the list

# Shuffle the list of files randomly
random.shuffle(n_files)
random.shuffle(ab_files)

#test
test_n_path ='/content/drive/MyDrive/Datasets/Car_Images_test/normal/'

# Extract the specified number of random files
for i in range(te_data_num[0]):
    random_file = n_files[i]
    path = tr_n_path+random_file
    n_files.remove(random_file)
    shutil.move(path, test_n_path)


#validaton after extract test image


In [ ]:
test_ab_path ='/content/drive/MyDrive/Datasets/Car_Images_test/abnormal/'

for i in range(te_data_num[1]):
    random_file = ab_files[i]
    path = tr_ab_path+random_file
    ab_files.remove(random_file)
    shutil.move(path, test_ab_path)


In [ ]:
# 추출 후 이미지 갯수 확인
print(len(os.listdir(test_n_path)))
print(len(os.listdir(test_ab_path)))
print(len(os.listdir(tr_ab_path)))
print(len(os.listdir(tr_n_path)))

60
61
242
242


#### 3) validation 셋 추출

In [ ]:
random.seed(2023)

# Get a list of all the files in the directory
n_files = os.listdir(tr_n_path)
ab_files = os.listdir(tr_ab_path)
# Choose a random file from the list

# Shuffle the list of files randomly
random.shuffle(n_files)
random.shuffle(ab_files)

val_n_path = '/content/drive/MyDrive/Datasets/Car_Images_validation/normal'
val_ab_path = '/content/drive/MyDrive/Datasets/Car_Images_validation/abnormal'

for i in range(val_data_num[0]):
    #validation- normal
    random_file = n_files[i]
    path = tr_n_path+random_file
    n_files.remove(random_file)
    shutil.move(path,val_n_path)

    #validation - abnormal
    random_file = ab_files[i]
    path = tr_ab_path+random_file
    ab_files.remove(random_file)
    shutil.move(path,val_ab_path)   

In [ ]:
# 추출 후 이미지 갯수 확인
print(len(os.listdir(val_ab_path)))
print(len(os.listdir(val_n_path)))

print(len(os.listdir(test_n_path)))
print(len(os.listdir(test_ab_path)))
print(len(os.listdir(tr_ab_path)))
print(len(os.listdir(tr_n_path)))



48
48
60
61
194
194


### (2) 데이터 복사 및 이동
- **세부요구사항**
    - 분할된 데이터를 복사 이동합니다.
        - 새로운 폴더에 저장하는 데이터로 "3.모델링I"에서 사용합니다.
        - 기존 폴더는 "4.모델링II > (1) Data Augmentation"에서 사용합니다.
    - Training set | Validation set | Test set의 데이터를 **새로운 폴더**에 복사하세요.
        - 새로운 폴더 명
            * copy_images/trainset
            * copy_images/validset
            * copy_images/testset
        - 새로운 폴더에는 normal, abnormal 파일 모두를 복사합니다. 
            * 파일을 구분하기 위해 abnormal 파일들은 파일명 앞에 접두사 'ab_'를 붙입시다.
        - os, shutil 모듈을 활용하세요.

#### 1) abnormal 파일 복사

* 복사하기 : shutil.copytree()

In [ ]:
os.mkdir('/content/drive/MyDrive/Datasets/copy_images')
# os.mkdir('/content/drive/MyDrive/Datasets/copy_images/trianset')
# os.mkdir('/content/drive/MyDrive/Datasets/copy_images/testset')
# os.mkdir('/content/drive/MyDrive/Datasets/copy_images/validset')

In [ ]:
import shutil
#train
tr_copy_path = '/content/drive/MyDrive/Datasets/copy_images/trainset/'
shutil.copytree(tr_ab_path, tr_copy_path)

'/content/drive/MyDrive/Datasets/copy_images/trainset/'

In [ ]:
#test
test_copy_path = '/content/drive/MyDrive/Datasets/copy_images/testset'
shutil.copytree(test_ab_path, test_copy_path)

'/content/drive/MyDrive/Datasets/copy_images/testset'

In [ ]:
#validation
val_copy_path = '/content/drive/MyDrive/Datasets/copy_images/validset'
shutil.copytree(val_ab_path, val_copy_path)

'/content/drive/MyDrive/Datasets/copy_images/validset'

* abnormal 이미지 이름의 접두어 "ab_" 붙이기 : os.rename

In [ ]:
#train
tr_ab_files = os.listdir(tr_copy_path)
for file in tr_ab_files:
    src =dst = os.path.join(tr_copy_path, file)
    dst = 'ab_'+file
    dst = os.path.join(tr_copy_path, dst)
    os.rename(src,dst)




In [ ]:
#test
test_ab_files = os.listdir(test_copy_path)
for file in test_ab_files:
    src =dst = os.path.join(test_copy_path, file)
    dst = 'ab_'+file
    dst = os.path.join(test_copy_path, dst)
    os.rename(src,dst)

In [ ]:
#validation
val_ab_files = os.listdir(val_copy_path)
for file in val_ab_files:
    src =dst = os.path.join(val_copy_path, file)
    dst = 'ab_'+file
    dst = os.path.join(val_copy_path, dst)
    os.rename(src,dst)

#### 2) normal 파일 복사

In [ ]:
from distutils.dir_util import copy_tree

copy_tree(tr_n_path, tr_copy_path)
copy_tree(test_n_path, test_copy_path)
copy_tree(val_n_path, val_copy_path)

['/content/drive/MyDrive/Datasets/copy_images/validset/DALLбдE 2023-03-10 18.52.44 - photo of a part of car.png',
 '/content/drive/MyDrive/Datasets/copy_images/validset/DALLбдE 2023-03-10 22.22.55 - photo of a part of car.png',
 '/content/drive/MyDrive/Datasets/copy_images/validset/DALLбдE 2023-03-10 22.37.34 - photo of a part of car.png',
 '/content/drive/MyDrive/Datasets/copy_images/validset/DALLбдE 2023-03-10 23.28.24 - photo of a part of car without blemish.png',
 '/content/drive/MyDrive/Datasets/copy_images/validset/DALLбдE 2023-03-10 23.31.15 - photo of a part of car without blemish.png',
 '/content/drive/MyDrive/Datasets/copy_images/validset/DALLбдE 2023-03-10 23.31.20 - photo of a part of car without blemish.png',
 '/content/drive/MyDrive/Datasets/copy_images/validset/DALLбдE 2023-03-10 23.31.56 - photo of a part of car without blemish.png',
 '/content/drive/MyDrive/Datasets/copy_images/validset/DALLбдE 2023-03-10 23.32.44 - photo of a part of car without blemish.png',
 '/conte

* 데이터 갯수 조회

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/Datasets/copy_images/trainset/')))
print(len(os.listdir('/content/drive/MyDrive/Datasets/copy_images/validset/')))
print(len(os.listdir('/content/drive/MyDrive/Datasets/copy_images/testset/')))

388
96
121


## 3.모델링 I
* **세부요구사항**
    * 모델링을 위한 데이터 구조 만들기
        * x : 이미지를 array로 변환합니다.
        * y : 이미지 갯수만큼 normal - 0, abnormal - 1 로 array를 만듭니다.
    * 모델을 최소 3개 이상 만들고 성능을 비교합니다.
        * 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
        * 전처리 과정에서 생성한 Validation set을 적절하게 사용하세요.
        * Early Stopping을 반드시 사용하세요.
            * 최적의 가중치를 모델에 적용하세요.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import os 
dataset_path  = '/content/drive/MyDrive/Datasets/'
from tensorflow.keras.preprocessing import image
import glob

from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, Dropout, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (1) X : image to array
- **세부요구사항**
    * 모델링을 위해서는 np.array 형태로 데이터셋을 만들어야 합니다.
    * Training set / Validation set / Test set의 X는 이미지 형태로 되어있습니다. 
    * 이미지 파일을 불러와 train, valid, test 각각 array 형태로 변환해 봅시다.
        * 각 폴더로 부터 이미지 목록을 만들고
        * 이미지 한장씩 적절한 크기로 로딩하여 (keras.utils.load_img)
            * 이미지가 너무 크면 학습시간이 많이 걸리고, 메모리 부족현상이 발생될 수 있습니다.
            * 이미지 크기를 280 * 280 * 3 이내의 크기를 설정하여 로딩하시오.
            * array로 변환 (keras.utils.img_to_array, np.expand_dims)
        * 데이터셋에 추가합니다.(데이터셋도 array)

#### 1) 이미지 목록 만들기
* train, validation, test 폴더로 부터 이미지 목록을 생성합니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 이미지 목록 저장
img_train_list = os.listdir(dataset_path+'copy_images/trainset/')
img_valid_list = os.listdir(dataset_path+'copy_images/validset/')
img_test_list = os.listdir(dataset_path+'copy_images/testset/')

In [ ]:
# 메모리, 처리시간을 위해서 이미지 크기 조정
img_size = 280 ## 사이즈 조정 가능

#### 2) 이미지들을 배열 데이터셋으로 만들기

In [ ]:
from tensorflow.keras.preprocessing import image
import glob

# train
images = []
# labels = []
tr_files = glob.glob('/content/drive/MyDrive/Datasets/copy_images/trainset/*')
for path in tr_files:
    img = image.load_img(path, target_size=(280,280) )
    img = image.img_to_array(img)
    
    images.append(img)
    # labels.append(names[path.split('/')[-2]])
    

x_train = np.array(images)
#x_train /=255
# labels_arr = np.array(labels)

print(x_train.shape)
# print(labels_arr.shape)





(388, 280, 280, 3)


In [ ]:
#test
images = []
# labels = []
test_files = glob.glob('/content/drive/MyDrive/Datasets/copy_images/testset/*')
for path in test_files:
    img = image.load_img(path, target_size=(280,280) )
    img = image.img_to_array(img)
    
    images.append(img)
    # labels.append(names[path.split('/')[-2]])
    

x_test = np.array(images)
#x_test /= 255
# labels_arr = np.array(labels)

print(x_test.shape)
# print(labels_arr.shape)

(121, 280, 280, 3)


In [ ]:
#validation
images = []
# labels = []
val_files = glob.glob('/content/drive/MyDrive/Datasets/copy_images/validset/*')
for path in val_files:
    img = image.load_img(path, target_size=(280,280) )
    img = image.img_to_array(img)
    
    images.append(img)
    # labels.append(names[path.split('/')[-2]])
    

x_val = np.array(images)
#x_val /=255
# labels_arr = np.array(labels)

print(x_val.shape)
# print(labels_arr.shape)

(96, 280, 280, 3)


### (2) y : 클래스 만들기
- **세부요구사항**
    - Training set / Validation set / Test set의 y를 생성합니다.
        - 각각 normal, abnormal 데이터의 갯수를 다시 확인하고
        - normal을 0, abnormal을 1로 지정합니다.

In [ ]:
# 데이터 갯수 확인
print( len(img_train_list) )
print( len([val for val in img_train_list if val.startswith('ab_')]) )
print('---')
print( len(img_valid_list) )
print( len([val for val in img_valid_list if val.startswith('ab_')]) )
print('---')
print( len(img_test_list) )
print( len([val for val in img_test_list if val.startswith('ab_')]) )

388
194
---
96
48
---
121
61


* y_train, y_valid, y_test 만들기
    * normal, abnormal 데이터의 갯수를 다시 확인하고 normal을 0, abnormal을 1로 지정합니다.

In [ ]:

labels=[]
for x in img_train_list:
    if x[:3] == 'ab_':
        labels.append(1)
    else:
        labels.append(0)


In [ ]:

y_train = np.array(labels)
#y_train = to_categorical(labels_tr,2)

In [ ]:
y_train.shape

(388,)

In [ ]:

labels=[]
for x in img_test_list:
    if x[:3] == 'ab_':
        labels.append(1)
    else:
        labels.append(0)
y_test= np.array(labels)

In [ ]:
labels=[]
for x in img_valid_list:
    if x[:3] == 'ab_':
        labels.append(1)
    else:
        labels.append(0)
y_val= np.array(labels)
#y_test = to_categorical(labels_val,2)

### (3) 모델1
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

In [1]:
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, Dropout, Concatenate

In [ ]:
clear_session()
#model
model = Sequential()

# 모델 조립
model.add(Input(shape=(280,280,3)))
model.add(Conv2D(
                filters=32,
                padding='same',
                 kernel_size=(3,3),
                 strides=(1,1),
                 activation='relu' 
                 ))
model.add(BatchNormalization())

model.add(Conv2D(
                filters=32,
                padding='same',
                 kernel_size=(3,3),
                 strides=(1,1),
                 activation='relu' 
                 ))

model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2)))

model.add(keras.layers.Dropout(0.25))

model.add(Conv2D(
                filters=64,
                padding='same',
                 kernel_size=(3,3),
                 strides=(1,1),
                 activation='relu' 
                 ))
model.add(BatchNormalization())

model.add(Conv2D(
                filters=64,
                padding='same',
                 kernel_size=(3,3),
                 strides=(1,1),
                 activation='relu' 
                 ))

model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2)))

model.add(keras.layers.Dropout(0.25))

model.add( Flatten() )
model.add( Dense(512, activation='relu') )
model.add( Dense(1, activation='sigmoid'))

# 컴파일! 
# y에 대한 전처리를 하지 않고도 자동으로 판단하고 cross entropy
# binary cross entropy를 쓰면 에러 
model.compile(loss=keras.losses.binary_crossentropy, metrics=['accuracy'],
              optimizer=keras.optimizers.Adam(lr=0.001) )
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 280, 280, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 280, 280, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 280, 280, 32)      9248      
                                                                 
 batch_normalization_1 (Batc  (None, 280, 280, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 140, 140, 32)     0         
 )                                                               
                                                        

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es  = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, restore_best_weights=True)


In [ ]:
x_train.shape, y_train.shape

((388, 280, 280, 3), (388,))

In [ ]:
hist = model.fit(x_train , y_train, epochs=1000, batch_size=16, validation_data=(x_val, y_val), verbose=1, callbacks=[es])

Epoch 1/1000
25/25 [==============================] - 11s 308ms/step - loss: 48.5745 - accuracy: 0.6521 - val_loss: 51.0954 - val_accuracy: 0.5208
Epoch 2/1000
25/25 [==============================] - 6s 249ms/step - loss: 1.3965 - accuracy: 0.7552 - val_loss: 42.8754 - val_accuracy: 0.5208
Epoch 3/1000
25/25 [==============================] - 6s 233ms/step - loss: 0.4000 - accuracy: 0.8273 - val_loss: 17.7756 - val_accuracy: 0.5729
Epoch 4/1000
25/25 [==============================] - 6s 251ms/step - loss: 0.2901 - accuracy: 0.8995 - val_loss: 7.2521 - val_accuracy: 0.6875
Epoch 5/1000
25/25 [==============================] - 6s 227ms/step - loss: 0.2322 - accuracy: 0.8840 - val_loss: 3.5290 - val_accuracy: 0.6667
Epoch 6/1000
25/25 [==============================] - 5s 192ms/step - loss: 0.1702 - accuracy: 0.9149 - val_loss: 3.7226 - val_accuracy: 0.7083
Epoch 7/1000
25/25 [==============================] - 5s 187ms/step - loss: 0.1615 - accuracy: 0.9459 - val_loss: 4.4625 - val_accu

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
performance_test = model.evaluate(x_test, y_test, batch_size=100)

print('Test Loss : {:.6f},  Test Accuracy : {:.3f}%'.format(performance_test[0], performance_test[1]*100))

2/2 [==============================] - 0s 45ms/step - loss: 3.5628 - accuracy: 0.7107
Test Loss : 3.562834,  Test Accuracy : 71.074%


In [ ]:
y_pred = model.predict(x_test)

for i in range(len(y_pred)):
    if y_pred[i] >= 0.5:
        y_pred[i] = 1
    else:
        y_pred[i] = 0

4/4 [==============================] - 1s 182ms/step


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.77      0.81        60
           1       0.79      0.89      0.84        61

    accuracy                           0.83       121
   macro avg       0.83      0.83      0.83       121
weighted avg       0.83      0.83      0.83       121



### (4) 모델2
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

# GoogLeNet

* **자기 자신의 미니버전을 만들어내는 것이 좋다.**
* 어렵다면, 일단 강사의 미니버전을 코드짜본다!

1. 인풋레이어를 제작한다.
2. 모든 히든레이어의 activation은 'relu'로 통일
3. 첫번째 히든레이어 : Conv 3*3, 32, same
4. BatchNorm
3. 두번째 히든레이어 : Conv 3*3, 64, same
4. BatchNorm
5. Maxpooling 2*2
6. 드랍아웃 (0.2)
2. 위의 드랍아웃 레이어에 연결되는 (왼쪽부터)첫번째 가지
    * Convolution : 필터개수 32개, 필터사이즈(1,1), 스트라이드(1,1), 패딩='same'
3. 위의 드랍아웃 레이어에 연결되는 두번째 가지
    * Convolution : 필터개수 64개, 필터사이즈(1,1), 스트라이드(1,1), 패딩='same'
    * Convolution : 필터개수 64개, 필터사이즈(3,3), 스트라이드 (1,1), 패딩='same'
4. 위의 드랍아웃 레이어에 연결되는 세번째 가지
    * Convolution : 필터개수 16개, 필터사이즈(1,1), 스트라이드(1,1), 패딩='same'
    * Convolution : 필터개수 16개, 필터사이즈(5,5), 스트라이드 (1,1), 패딩='same'
5. 위의 드랍아웃 레이어에 연결되는 네번째 가지
    * MaxPooling : 사이즈(3,3), 스트라이드(1,1), 패딩='same'
    * Convolution : 필터개수 16개, 필터사이즈(1,1), 패딩='same'
6. Concat.(채널기준으로 통합함. axis=-1 또는 axis=3)
7. GlobalAveragePooling2D 레이어
13. 아웃풋레이어 



In [ ]:
clear_session()
#model
# 인풋레이어를 제작한다.
il = Input(shape=(280,280,3))
# 모든 히든레이어의 activation은 'relu'로 통일
# 첫번째 히든레이어 : Conv 3*3, 32, same
hl = Conv2D(filters=32,kernel_size=(3,3),padding='same', activation='relu')(il)
# BatchNorm
hl = BatchNormalization()(hl)
# 두번째 히든레이어 : Conv 3*3, 64, same
hl = Conv2D(filters=64, kernel_size=(3,3), padding='same',  activation='relu')(hl)
# BatchNorm
hl = BatchNormalization()(hl)
# Maxpooling 2*2
hl = MaxPool2D(pool_size=(2,2))(hl)
# 드랍아웃 (0.2)
path1 = Dropout(0.25)(hl)
# 위의 드랍아웃 레이어에 연결되는 (왼쪽부터)첫번째 가지
# Convolution : 필터개수 32개, 필터사이즈(1,1), 스트라이드(1,1), 패딩='same'
path1_1 = Conv2D(filters=32, kernel_size=(1,1),padding='same',  activation='relu')(path1)
# 위의 드랍아웃 레이어에 연결되는 두번째 가지
# Convolution : 필터개수 64개, 필터사이즈(1,1), 스트라이드(1,1), 패딩='same'
path1_2 = Conv2D(filters=64, kernel_size=(1,1),padding='same',  activation='relu')(path1)
# Convolution : 필터개수 64개, 필터사이즈(3,3), 스트라이드 (1,1), 패딩='same'
path1_2 = Conv2D(filters=64, kernel_size=(3,3),padding='same',  activation='relu')(path1_2)
# 위의 드랍아웃 레이어에 연결되는 세번째 가지
# Convolution : 필터개수 16개, 필터사이즈(1,1), 스트라이드(1,1), 패딩='same'
# Convolution : 필터개수 16개, 필터사이즈(5,5), 스트라이드 (1,1), 패딩='same'
path1_3 = Conv2D(filters=16, kernel_size=(1,1),padding='same',  activation='relu')(path1)
path1_3 = Conv2D(filters=16, kernel_size=(5,5),padding='same',  activation='relu')(path1_3)
# 위의 드랍아웃 레이어에 연결되는 네번째 가지
# MaxPooling : 사이즈(3,3), 스트라이드(1,1), 패딩='same'
# Convolution : 필터개수 16개, 필터사이즈(1,1), 패딩='same'
path1_4 = MaxPool2D(pool_size=(3,3),strides=(1,1), padding='same')(path1)
path1_4 = Conv2D(filters=16, kernel_size=(1,1),padding='same',  activation='relu')(path1_4)
# Concat.(채널기준으로 통합함. axis=-1 또는 axis=3)
concate = Concatenate()([path1_1, path1_2, path1_3, path1_4])
# GlobalAveragePooling2D 레이어
hl = GlobalAveragePooling2D()(concate)
# 아웃풋레이어
ol = Dense(1,activation='sigmoid')(hl)

model = Model(il,ol)
model.compile(loss=keras.losses.binary_crossentropy, metrics=['accuracy'], optimizer='adam')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 280, 280, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 280, 280, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 280, 280, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=7,
                   verbose=1,
                   restore_best_weights=True
                   )

In [ ]:
hist = model.fit(x_train , y_train, epochs=1000, batch_size=16, validation_data=(x_val, y_val), verbose=1, callbacks=[es])

Epoch 1/1000
25/25 [==============================] - 92s 246ms/step - loss: 0.5594 - accuracy: 0.7268 - val_loss: 0.7041 - val_accuracy: 0.5000
Epoch 2/1000
25/25 [==============================] - 6s 221ms/step - loss: 0.4799 - accuracy: 0.7912 - val_loss: 0.7557 - val_accuracy: 0.5000
Epoch 3/1000
25/25 [==============================] - 6s 221ms/step - loss: 0.4714 - accuracy: 0.7732 - val_loss: 0.8431 - val_accuracy: 0.5000
Epoch 4/1000
25/25 [==============================] - 6s 224ms/step - loss: 0.4586 - accuracy: 0.7964 - val_loss: 0.9505 - val_accuracy: 0.5000
Epoch 5/1000
25/25 [==============================] - 6s 222ms/step - loss: 0.4112 - accuracy: 0.8273 - val_loss: 1.0651 - val_accuracy: 0.5000
Epoch 6/1000
25/25 [==============================] - 6s 223ms/step - loss: 0.4333 - accuracy: 0.8067 - val_loss: 1.2168 - val_accuracy: 0.5000
Epoch 7/1000
25/25 [==============================] - 6s 220ms/step - loss: 0.4031 - accuracy: 0.8247 - val_loss: 1.3124 - val_accuracy

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
performance_test = model.evaluate(x_test, y_test, batch_size=100)

print('Test Loss : {:.6f},  Test Accuracy : {:.3f}%'.format(performance_test[0], performance_test[1]*100))

In [ ]:
y_pred = model.predict(x_test)

for i in range(len(y_pred)):
    if y_pred[i] >= 0.5:
        y_pred[i] = 1
    else:
        y_pred[i] = 0

In [ ]:
print(classification_report(y_test,y_pred))

### (5) 모델3
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계 - 모델2에 data argumentation 적용

In [ ]:
idg = ImageDataGenerator(rotation_range=20, #이미지 회전
                         width_shift_range=0.1, #이미지 좌우 이동
                         height_shift_range=0.1, #이미지 상하 이송
                         zoom_range=0.1,#확대 축소 범위
                         shear_range=0.1, 
                         horizontal_flip=True,
                         vertical_flip=True,
                         fill_mode='nearest')

idg.fit(x_train)

In [ ]:
train_idg = idg.flow(x_train, y_train, batch_size=16)

In [ ]:
clear_session()
#model
# 인풋레이어를 제작한다.
il = Input(shape=(280,280,3))
# 모든 히든레이어의 activation은 'relu'로 통일
# 첫번째 히든레이어 : Conv 3*3, 32, same
hl = Conv2D(filters=32,kernel_size=(3,3),padding='same', activation='relu')(il)
# BatchNorm
hl = BatchNormalization()(hl)
# 두번째 히든레이어 : Conv 3*3, 64, same
hl = Conv2D(filters=64, kernel_size=(3,3), padding='same',  activation='relu')(hl)
# BatchNorm
hl = BatchNormalization()(hl)
# Maxpooling 2*2
hl = MaxPool2D(pool_size=(2,2))(hl)
# 드랍아웃 (0.2)
path1 = Dropout(0.25)(hl)
# 위의 드랍아웃 레이어에 연결되는 (왼쪽부터)첫번째 가지
# Convolution : 필터개수 32개, 필터사이즈(1,1), 스트라이드(1,1), 패딩='same'
path1_1 = Conv2D(filters=32, kernel_size=(1,1),padding='same',  activation='relu')(path1)
# 위의 드랍아웃 레이어에 연결되는 두번째 가지
# Convolution : 필터개수 64개, 필터사이즈(1,1), 스트라이드(1,1), 패딩='same'
path1_2 = Conv2D(filters=64, kernel_size=(1,1),padding='same',  activation='relu')(path1)
# Convolution : 필터개수 64개, 필터사이즈(3,3), 스트라이드 (1,1), 패딩='same'
path1_2 = Conv2D(filters=64, kernel_size=(3,3),padding='same',  activation='relu')(path1_2)
# 위의 드랍아웃 레이어에 연결되는 세번째 가지
# Convolution : 필터개수 16개, 필터사이즈(1,1), 스트라이드(1,1), 패딩='same'
# Convolution : 필터개수 16개, 필터사이즈(5,5), 스트라이드 (1,1), 패딩='same'
path1_3 = Conv2D(filters=16, kernel_size=(1,1),padding='same',  activation='relu')(path1)
path1_3 = Conv2D(filters=16, kernel_size=(5,5),padding='same',  activation='relu')(path1_3)
# 위의 드랍아웃 레이어에 연결되는 네번째 가지
# MaxPooling : 사이즈(3,3), 스트라이드(1,1), 패딩='same'
# Convolution : 필터개수 16개, 필터사이즈(1,1), 패딩='same'
path1_4 = MaxPool2D(pool_size=(3,3),strides=(1,1), padding='same')(path1)
path1_4 = Conv2D(filters=16, kernel_size=(1,1),padding='same',  activation='relu')(path1_4)
# Concat.(채널기준으로 통합함. axis=-1 또는 axis=3)
concate = Concatenate()([path1_1, path1_2, path1_3, path1_4])
# GlobalAveragePooling2D 레이어
hl = GlobalAveragePooling2D()(concate)
# 아웃풋레이어
ol = Dense(1,activation='sigmoid')(hl)

model = Model(il,ol)
model.compile(loss=keras.losses.binary_crossentropy, metrics=['accuracy'], optimizer='adam')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 280, 280, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 280, 280, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 280, 280, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=7,
                   verbose=1,
                   restore_best_weights=True)
# hist = model.fit(train_idg, validation_data=(x_val, y_val)
#                  ,epochs=10000, verbose=1, callbacks=[es])

In [ ]:
hist = model.fit(train_idg, validation_data=(x_val, y_val)
                 ,epochs=10000, verbose=1, callbacks=[es])

Epoch 1/10000
25/25 [==============================] - 12s 354ms/step - loss: 0.6760 - accuracy: 0.6572 - val_loss: 1.0818 - val_accuracy: 0.5000
Epoch 2/10000
25/25 [==============================] - 9s 351ms/step - loss: 0.4928 - accuracy: 0.7680 - val_loss: 1.6395 - val_accuracy: 0.5000
Epoch 3/10000
25/25 [==============================] - 8s 311ms/step - loss: 0.4631 - accuracy: 0.7809 - val_loss: 1.1988 - val_accuracy: 0.5000
Epoch 4/10000
25/25 [==============================] - 9s 350ms/step - loss: 0.4788 - accuracy: 0.7861 - val_loss: 1.3579 - val_accuracy: 0.5000
Epoch 5/10000
25/25 [==============================] - 9s 353ms/step - loss: 0.4653 - accuracy: 0.7912 - val_loss: 0.8106 - val_accuracy: 0.6875
Epoch 6/10000
25/25 [==============================] - 8s 309ms/step - loss: 0.4969 - accuracy: 0.7680 - val_loss: 0.7309 - val_accuracy: 0.6875
Epoch 7/10000
25/25 [==============================] - 9s 354ms/step - loss: 0.4441 - accuracy: 0.8015 - val_loss: 0.8073 - val_a

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
performance_test = model.evaluate(x_test, y_test, batch_size=100)

print('Test Loss : {:.6f},  Test Accuracy : {:.3f}%'.format(performance_test[0], performance_test[1]*100))

2/2 [==============================] - 0s 55ms/step - loss: 0.5526 - accuracy: 0.7438
Test Loss : 0.552594,  Test Accuracy : 74.380%


In [ ]:
y_pred = model.predict(x_test)

for i in range(len(y_pred)):
    if y_pred[i] >= 0.5:
        y_pred[i] = 1
    else:
        y_pred[i] = 0

4/4 [==============================] - 0s 87ms/step


In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.67      0.97      0.79        60
           1       0.94      0.52      0.67        61

    accuracy                           0.74       121
   macro avg       0.80      0.75      0.73       121
weighted avg       0.81      0.74      0.73       121



### (6) 모델4
- **세부요구사항*
   - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계 - minmax scaling + ResNet

In [ ]:
max_num, min_num = x_train.max(), x_train.min()
x_train = (x_train-min_num)/(max_num-min_num)
x_val = (x_val-min_num)/(max_num-min_num)
x_test = (x_test-min_num)/(max_num-min_num)

In [ ]:
# idg = ImageDataGenerator(rotation_range=20,
#                          width_shift_range=0.1,
#                          height_shift_range=0.1,
#                          zoom_range=0.1,
#                          shear_range=0.1,
#                          horizontal_flip=True,
#                          vertical_flip=True)

# idg.fit(x_train)

In [ ]:
# train_idg = idg.flow(x_train, y_train)

In [ ]:
clear_session()
#model
# 인풋레이어를 제작한다.
il = Input(shape=(280,280,3))
# 모든 히든레이어의 activation은 'relu'로 통일
# 첫번째 히든레이어 : Conv 3*3, 32, same
hl = Conv2D(filters=32,kernel_size=(3,3),padding='same', activation='relu')(il)
# BatchNorm
hl = BatchNormalization()(hl)
# 두번째 히든레이어 : Conv 3*3, 64, same
hl = Conv2D(filters=64, kernel_size=(3,3), padding='same',  activation='relu')(hl)
# BatchNorm
hl = BatchNormalization()(hl)
# Maxpooling 2*2
hl = MaxPool2D(pool_size=(2,2))(hl)
# 드랍아웃 (0.2)
path1 = Dropout(0.25)(hl)
# 위의 드랍아웃 레이어에 연결되는 (왼쪽부터)첫번째 가지
# Convolution : 필터개수 32개, 필터사이즈(1,1), 스트라이드(1,1), 패딩='same'
path1_1 = Conv2D(filters=32, kernel_size=(1,1),padding='same',  activation='relu')(path1)
# 위의 드랍아웃 레이어에 연결되는 두번째 가지
# Convolution : 필터개수 64개, 필터사이즈(1,1), 스트라이드(1,1), 패딩='same'
path1_2 = Conv2D(filters=64, kernel_size=(1,1),padding='same',  activation='relu')(path1)
# Convolution : 필터개수 64개, 필터사이즈(3,3), 스트라이드 (1,1), 패딩='same'
path1_2 = Conv2D(filters=64, kernel_size=(3,3),padding='same',  activation='relu')(path1_2)
# 위의 드랍아웃 레이어에 연결되는 세번째 가지
# Convolution : 필터개수 16개, 필터사이즈(1,1), 스트라이드(1,1), 패딩='same'
# Convolution : 필터개수 16개, 필터사이즈(5,5), 스트라이드 (1,1), 패딩='same'
path1_3 = Conv2D(filters=16, kernel_size=(1,1),padding='same',  activation='relu')(path1)
path1_3 = Conv2D(filters=16, kernel_size=(5,5),padding='same',  activation='relu')(path1_3)
# 위의 드랍아웃 레이어에 연결되는 네번째 가지
# MaxPooling : 사이즈(3,3), 스트라이드(1,1), 패딩='same'
# Convolution : 필터개수 16개, 필터사이즈(1,1), 패딩='same'
path1_4 = MaxPool2D(pool_size=(3,3),strides=(1,1), padding='same')(path1)
path1_4 = Conv2D(filters=16, kernel_size=(1,1),padding='same',  activation='relu')(path1_4)
# Concat.(채널기준으로 통합함. axis=-1 또는 axis=3)
concate = Concatenate()([path1_1, path1_2, path1_3, path1_4])
# GlobalAveragePooling2D 레이어
hl = GlobalAveragePooling2D()(concate)
# 아웃풋레이어
ol = Dense(1,activation='sigmoid')(hl)

model = Model(il,ol)
model.compile(loss=keras.losses.binary_crossentropy, metrics=['accuracy'], optimizer='adam')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 280, 280, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 280, 280, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 280, 280, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=7,
                   verbose=1,
                   restore_best_weights=True)

In [ ]:
hist = model.fit(x_train, y_train,validation_data=(x_val, y_val), batch_size=16,
                 epochs=10000, verbose=1, callbacks=[es])

Epoch 1/10000
25/25 [==============================] - 6s 230ms/step - loss: 0.5134 - accuracy: 0.7577 - val_loss: 1.1070 - val_accuracy: 0.5000
Epoch 2/10000
25/25 [==============================] - 5s 219ms/step - loss: 0.4827 - accuracy: 0.7680 - val_loss: 1.1079 - val_accuracy: 0.5000
Epoch 3/10000
25/25 [==============================] - 5s 219ms/step - loss: 0.5284 - accuracy: 0.7500 - val_loss: 0.8313 - val_accuracy: 0.5000
Epoch 4/10000
25/25 [==============================] - 5s 217ms/step - loss: 0.4596 - accuracy: 0.7861 - val_loss: 0.6997 - val_accuracy: 0.5000
Epoch 5/10000
25/25 [==============================] - 5s 214ms/step - loss: 0.4473 - accuracy: 0.7964 - val_loss: 0.7297 - val_accuracy: 0.5000
Epoch 6/10000
25/25 [==============================] - 5s 219ms/step - loss: 0.4315 - accuracy: 0.8067 - val_loss: 1.1911 - val_accuracy: 0.5000
Epoch 7/10000
25/25 [==============================] - 5s 218ms/step - loss: 0.4443 - accuracy: 0.8015 - val_loss: 1.4124 - val_ac

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
performance_test = model.evaluate(x_test, y_test, batch_size=100)

print('Test Loss : {:.6f},  Test Accuracy : {:.3f}%'.format(performance_test[0], performance_test[1]*100))

2/2 [==============================] - 1s 51ms/step - loss: 0.6908 - accuracy: 0.4959
Test Loss : 0.690752,  Test Accuracy : 49.587%


In [ ]:
# performance_test = model.evaluate(x_test, y_test, batch_size=100)

# print('Test Loss : {:.6f},  Test Accuracy : {:.3f}%'.format(performance_test[0], performance_test[1]*100))

y_pred = model.predict(x_test)

for i in range(len(y_pred)):
    if y_pred[i] >= 0.5:
        y_pred[i] = 1
    else:
        y_pred[i] = 0
# print(classification_report(y_test,y_pred))

4/4 [==============================] - 0s 97ms/step


In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.50      1.00      0.66        60
           1       0.00      0.00      0.00        61

    accuracy                           0.50       121
   macro avg       0.25      0.50      0.33       121
weighted avg       0.25      0.50      0.33       121



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 4.모델링 II
* **세부요구사항**
    - 성능을 높이기 위해서 다음의 두가지를 시도해 봅시다.
        - Data Augmentation을 통해 데이터를 증가 시킵니다.
            - ImageDataGenerator를 사용합니다.
        - 사전 학습된 모델(Transfer Learning)을 가져다 사용해 봅시다.
            - VGG16(이미지넷)을 사용해 봅시다.

### (1) Data Augmentation
- **세부요구사항**
    * 모델 학습에 이용할 이미지 데이터를 증강시키세요.
    * Keras의 ImageDataGenerator를 이용
        - [ImageDataGenerator document](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

    * image generator를 이용하여 학습
        * 모델 구조는 이미 생성한 1,2,3 중 하나를 선택하여 학습


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_size = 280 ## 사이즈 조정 가능

train_path = dataset_path+'Car_Images_train/'
valid_path = dataset_path+'Car_Images_validation/'
test_path = dataset_path+'Car_Images_test/'

#### 1) ImageGenerator 생성
* ImageDataGenerator 함수 사용
    * 주요 옵션
        * rotation_range: 무작위 회전을 적용할 각도 범위
        * zoom_range: 무작위 줌을 적용할 범위 [1-zoom_range, 1+zoom_range]
        * horizontal_flip: 무작위 좌우반전을 적용할지 여부
        * vertical_flip: 무작위 상하반전을 적용할지 여부
        * rescale: 텐서의 모든 값을 rescale 값으로 나누어줌 (이 경우에는 255로 나누어서 0~1사이의 값으로 변경)

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=30,
    zoom_range = 0.2,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    vertical_flip = True,
    rescale=1./255,
)

valid_datagen =  ImageDataGenerator(
    rotation_range=30,
    zoom_range = 0.2,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    vertical_flip = True,
    rescale=1./255,
)


#### 2) 경로로 부터 이미지 불러 올 준비
* .flow_from_directory 이용
    * 디렉토리에서 이미지를 가져와서 데이터 증강을 적용하고 batch 단위로 제공하는 generator를 생성합니다.
    * 이미지를 불러올 때 target_size로 크기를 맞추고, 
    * class_mode로 이진 분류(binary)를 수행하도록 지정합니다.


In [ ]:
train_generator = train_datagen.flow_from_directory(directory=train_path,
                                                    target_size=(280,280),
                                                    class_mode='binary',
                                                    batch_size=16,
                                                    shuffle=True
                                                    )


valid_generator =  valid_datagen.flow_from_directory(directory=valid_path,
                                                    target_size=(280,280),
                                                    class_mode='binary',
                                                    batch_size=16,
                                                    shuffle=True
                                                    )


Found 388 images belonging to 2 classes.
Found 96 images belonging to 2 classes.


In [ ]:
test_datagen =  ImageDataGenerator(
    rescale=1./255,
)
test_generator =  valid_datagen.flow_from_directory(directory=test_path,                                                    
                                                    target_size=(280,280),
                                                    class_mode='binary',
                                                    batch_size=16,
                                                    shuffle=True )

Found 121 images belonging to 2 classes.


#### 3) 학습
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 train_generator 이용. 
    - validation_data = valid_generator 지정
    - Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

* 구조 설계

In [ ]:
clear_session()
#model
# 인풋레이어를 제작한다.
il = Input(shape=(280,280,3))
# 모든 히든레이어의 activation은 'relu'로 통일
# 첫번째 히든레이어 : Conv 3*3, 32, same
hl = Conv2D(filters=32,kernel_size=(3,3),padding='same', activation='relu')(il)
# BatchNorm
hl = BatchNormalization()(hl)
# 두번째 히든레이어 : Conv 3*3, 64, same
hl = Conv2D(filters=64, kernel_size=(3,3), padding='same',  activation='relu')(hl)
# BatchNorm
hl = BatchNormalization()(hl)
# Maxpooling 2*2
hl = MaxPool2D(pool_size=(2,2))(hl)
# 드랍아웃 (0.2)
path1 = Dropout(0.25)(hl)
# 위의 드랍아웃 레이어에 연결되는 (왼쪽부터)첫번째 가지
# Convolution : 필터개수 32개, 필터사이즈(1,1), 스트라이드(1,1), 패딩='same'
path1_1 = Conv2D(filters=32, kernel_size=(1,1),padding='same',  activation='relu')(path1)
# 위의 드랍아웃 레이어에 연결되는 두번째 가지
# Convolution : 필터개수 64개, 필터사이즈(1,1), 스트라이드(1,1), 패딩='same'
path1_2 = Conv2D(filters=64, kernel_size=(1,1),padding='same',  activation='relu')(path1)
# Convolution : 필터개수 64개, 필터사이즈(3,3), 스트라이드 (1,1), 패딩='same'
path1_2 = Conv2D(filters=64, kernel_size=(3,3),padding='same',  activation='relu')(path1_2)
# 위의 드랍아웃 레이어에 연결되는 세번째 가지
# Convolution : 필터개수 16개, 필터사이즈(1,1), 스트라이드(1,1), 패딩='same'
# Convolution : 필터개수 16개, 필터사이즈(5,5), 스트라이드 (1,1), 패딩='same'
path1_3 = Conv2D(filters=16, kernel_size=(1,1),padding='same',  activation='relu')(path1)
path1_3 = Conv2D(filters=16, kernel_size=(5,5),padding='same',  activation='relu')(path1_3)
# 위의 드랍아웃 레이어에 연결되는 네번째 가지
# MaxPooling : 사이즈(3,3), 스트라이드(1,1), 패딩='same'
# Convolution : 필터개수 16개, 필터사이즈(1,1), 패딩='same'
path1_4 = MaxPool2D(pool_size=(3,3),strides=(1,1), padding='same')(path1)
path1_4 = Conv2D(filters=16, kernel_size=(1,1),padding='same',  activation='relu')(path1_4)
# Concat.(채널기준으로 통합함. axis=-1 또는 axis=3)
concate = Concatenate()([path1_1, path1_2, path1_3, path1_4])
# GlobalAveragePooling2D 레이어
hl = GlobalAveragePooling2D()(concate)
# 아웃풋레이어
ol = Dense(1,activation='sigmoid')(hl)

model = Model(il,ol)
model.compile(loss=keras.losses.binary_crossentropy, metrics=['accuracy'], optimizer=keras.optimizers.Adam() )
#model.summary()

* 학습
    * EarlyStopping 설정하기
    * 학습 데이터에 train_generator, validation_data=valid_generator 사용

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=7,
                   verbose=1,
                   restore_best_weights=True)
hist = model.fit(train_generator, validation_data=valid_generator
                 ,epochs=10000, verbose=1, callbacks=[es])

Epoch 1/10000
25/25 [==============================] - 36s 1s/step - loss: 0.5604 - accuracy: 0.6753 - val_loss: 0.7104 - val_accuracy: 0.5000
Epoch 2/10000
25/25 [==============================] - 29s 1s/step - loss: 0.4922 - accuracy: 0.7706 - val_loss: 0.7949 - val_accuracy: 0.5000
Epoch 3/10000
25/25 [==============================] - 29s 1s/step - loss: 0.4826 - accuracy: 0.7706 - val_loss: 0.9227 - val_accuracy: 0.5000
Epoch 4/10000
25/25 [==============================] - ETA: 0s - loss: 0.4732 - accuracy: 0.7912

KeyboardInterrupt: ignored

#### 4) 성능 평가
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
performance_test = model.evaluate(x_test,y_test,  batch_size=100)

print('Test Loss : {:.6f},  Test Accuracy : {:.3f}%'.format(performance_test[0], performance_test[1]*100))

# y_pred = model.predict(test_ds)

# for i in range(len(y_pred)):
#     if y_pred[i] >= 0.5:
#         y_pred[i] = 1
#     else:
#         y_pred[i] = 0
# print(classification_report(single_y_test,single_y_pred))

### (2) Transfer Learning
- **세부요구사항**
    * VGG16 모델은 1000개의 클래스를 분류하는 데 사용된 ImageNet 데이터셋을 기반으로 사전 학습된 가중치를 가지고 있습니다. 
        * 따라서 이 모델은 이미지 분류 문제에 대한 높은 성능을 보입니다.
        * 이 모델은 보통 전이학습(transfer learning)에서 기본적으로 사용되며, 특히 대규모 데이터셋이 없을 때는 기본 모델로 사용되어 fine-tuning을 수행합니다.
    * VGG16 함수로 부터 base_model 저장


In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B2
from tensorflow.keras.applications.efficientnet_v2 import  preprocess_input
from tensorflow.keras.applications.efficientnet_v2 import decode_predictions
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

#### 1) VGG16 불러와서 저장하기
* include_top=False로 설정하여 분류기를 제외하고 미리 학습된 가중치 imagenet을 로드합니다.
* .trainable을 True로 설정하여 모델의 모든 레이어들이 fine-tuning에 대해 업데이트되도록 합니다.


In [ ]:
clear_session()
base_model = VGG16( weights='imagenet',include_top=False, input_shape=(280,280,3))
new_output = GlobalAveragePooling2D()(base_model.output)
new_output = Dense(1, activation='sigmoid')(new_output)

model = keras.models.Model(base_model.inputs, new_output)




In [ ]:
for idx, layer in enumerate(model.layers):
    layer.trainable =True

#### 2) VGG16과 연결한 구조 설계
* VGG16을 불러와서 Flatten, Dense 등으로 레이어 연결하기

#### 3) 학습
- **세부요구사항**
    - 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
    - 데이터
        * Image Generator를 연결하거나
        * 기존 train, validation 셋을 이용해도 됩니다.
        - Early Stopping을 반드시 사용하세요.
        - 최적의 가중치를 모델에 적용하세요.

In [ ]:
model.compile(loss='binary_crossentropy', metrics=['accuracy'],
             optimizer=keras.optimizers.Adam(learning_rate=0.0001) )

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
lr_reduction = ReduceLROnPlateau(monitor='val_loss',
                                 patience=4,
                                 verbose=1,
                                 factor=0.5,
                                 min_lr=0.000001)

In [ ]:
hist = model.fit(train_generator,
                 epochs=1000, validation_data=valid_generator,
                 verbose=1, callbacks=[es, lr_reduction] )

Epoch 1/1000
25/25 [==============================] - 35s 1s/step - loss: 0.6081 - accuracy: 0.6959 - val_loss: 0.5120 - val_accuracy: 0.7500 - lr: 1.0000e-04
Epoch 2/1000
25/25 [==============================] - 31s 1s/step - loss: 0.4503 - accuracy: 0.8222 - val_loss: 0.3565 - val_accuracy: 0.8750 - lr: 1.0000e-04
Epoch 3/1000
25/25 [==============================] - 30s 1s/step - loss: 0.3701 - accuracy: 0.8454 - val_loss: 0.2875 - val_accuracy: 0.8438 - lr: 1.0000e-04
Epoch 4/1000
25/25 [==============================] - 30s 1s/step - loss: 0.3004 - accuracy: 0.8814 - val_loss: 0.2571 - val_accuracy: 0.9167 - lr: 1.0000e-04
Epoch 5/1000
25/25 [==============================] - 31s 1s/step - loss: 0.2913 - accuracy: 0.9072 - val_loss: 0.2248 - val_accuracy: 0.9167 - lr: 1.0000e-04
Epoch 6/1000
25/25 [==============================] - 30s 1s/step - loss: 0.2437 - accuracy: 0.9124 - val_loss: 0.2131 - val_accuracy: 0.9167 - lr: 1.0000e-04
Epoch 7/1000
25/25 [==========================

#### 4) 성능 평가

In [ ]:
# from tensorflow.keras.preprocessing import image_dataset_from_directory
# test_ds = image_dataset_from_directory(
#     directory=test_path,
#     labels='inferred',
#     label_mode='categorical',
#     batch_size=32,
#     image_size=(280,280)
# )

In [ ]:

performance_test = model.evaluate(test_generator,  batch_size=100)

print('Test Loss : {:.6f},  Test Accuracy : {:.3f}%'.format(performance_test[0], performance_test[1]*100))

# for i in range(len(y_pred)):
#     if y_pred[i] >= 0.5:
#         y_pred[i] = 1
#     else:
#         y_pred[i] = 0
# print(classification_report(single_y_test,single_y_pred))

8/8 [==============================] - 8s 934ms/step - loss: 0.0605 - accuracy: 0.9752
Test Loss : 0.060507,  Test Accuracy : 97.521%
4/4 [==============================] - 4s 211ms/step


In [ ]:
y_pred = model.predict(test_generator)

8/8 [==============================] - 7s 934ms/step


In [ ]:
print(classification_report(test_generator, y_pred))

In [ ]:
test_generator.image_shape

(280, 280, 3)

In [ ]:
y_pred.shape

(121, 1)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# test_generator에 관한 전처리
y_pred = model.predict(test_generator)
y_pred = y_pred.flatten()
y_pred = np.where(y_pred > 0.5, 1, 0)
y_test = test_generator.classes

print(classification_report(y_test, y_pred))

8/8 [==============================] - 7s 911ms/step
              precision    recall  f1-score   support

           0       0.48      0.49      0.48        61
           1       0.47      0.45      0.46        60

    accuracy                           0.47       121
   macro avg       0.47      0.47      0.47       121
weighted avg       0.47      0.47      0.47       121



In [ ]:
print(classification_report(y_test,y_pred))

ValueError: ignored